In [ ]:
import os
import face_recognition
import numpy as np
import cv2
from flask import Flask, send_file
from PIL import Image, ImageDraw

app = Flask(__name__)

@app.route('/face-recognition', methods=['POST'])
def recognition_face():
    jit_image = face_recognition.load_image_file("jit.jpg")
    jit_face_encoding = face_recognition.face_encodings(jit_image)[0]

    # Load a second sample picture and learn how to recognize it.
    Him_image = face_recognition.load_image_file("Him.jpg")
    Him_face_encoding = face_recognition.face_encodings(Him_image)[0]

    neer_image = face_recognition.load_image_file("neer.jpg")
    neer_face_encoding = face_recognition.face_encodings(neer_image)[0]

    ari_image = face_recognition.load_image_file("ari.jpg")
    ari_face_encoding = face_recognition.face_encodings(ari_image)[0]

    agar_image = face_recognition.load_image_file("agar.jpg")
    agar_face_encoding = face_recognition.face_encodings(agar_image)[0]

    a1_image = face_recognition.load_image_file("a1.jpg")
    a1_face_encoding = face_recognition.face_encodings(a1_image)[0]

    # Create arrays of known face encodings and their names
    known_face_encodings = [
        jit_face_encoding,
        Him_face_encoding,
        neer_face_encoding,
        ari_face_encoding,
        agar_face_encoding,
        a1_face_encoding

    ]
    known_face_names = [
        "Jitesh Kumar Gautam",
        "Himanshu Sachdeva",
        "Neeraj Arora",
        "Arihant Jain",
        "Abhishek Agarwal",
        "Abhishek Paliwal"
    ]
    #print('Learned encoding for', len(known_face_encodings), 'images.')

    key = cv2. waitKey(1)
    webcam = cv2.VideoCapture(0)
    while True:
        try:
            check, frame = webcam.read()
            cv2.imshow("Capturing", frame)
            key = cv2.waitKey(1)
            if key == ord('s'): 
                cv2.imwrite(filename='saved_img.jpg', img=frame)
                webcam.release()
                img_new = cv2.imread('saved_img.jpg', cv2.IMREAD_GRAYSCALE)
                img_new = cv2.imshow("Captured Image", img_new)
                cv2.waitKey(1650)
                cv2.destroyAllWindows()
                img_ = cv2.imread('saved_img.jpg', cv2.IMREAD_COLOR)
                img_resized = cv2.imwrite(filename='saved_img-final.jpg', img=img_)

                break
            elif key == ord('q'):
                webcam.release()
                cv2.destroyAllWindows()
                break

        except(KeyboardInterrupt):
            webcam.release()
            cv2.destroyAllWindows()
            break

    # Load an image with an unknown face
    unknown_image = face_recognition.load_image_file("saved_img-final.jpg")

    # Find all the faces and face encodings in the unknown image
    face_locations = face_recognition.face_locations(unknown_image)
    face_encodings = face_recognition.face_encodings(unknown_image, face_locations)

    pil_image = Image.fromarray(unknown_image)
    # Create a Pillow ImageDraw Draw instance to draw with
    draw = ImageDraw.Draw(pil_image)

    # Loop through each face found in the unknown image
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # See if the face is a match for the known face(s)
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)

        name = "Unknown"

        # Or instead, use the known face with the smallest distance to the new face
        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        best_match_index = np.argmin(face_distances)
        if matches[best_match_index]:
            name = known_face_names[best_match_index]

        # Draw a box around the face using the Pillow module
        draw.rectangle(((left, top), (right, bottom)), outline=(0, 0, 255))

        # Draw a label with a name below the face
        text_width, text_height = draw.textsize(name)
        draw.rectangle(((left, bottom - text_height - 10), (right, bottom)), fill=(0, 0, 255), outline=(0, 0, 255))
        draw.text((left + 6, bottom - text_height - 5), name, fill=(255, 255, 255, 255))


    # Remove the drawing library from memory as per the Pillow docs
    del draw
    # Display the resulting image
    pil_image.save("user.jpg")
    return send_file("user.jpg", mimetype='image/gif')

port = os.getenv('PORT', 5000)
if __name__ == '__main__':
    app.run(host="0.0.0.0", port=port)